In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Define the parameters
bbox = "-91.5,28,-88.75,30"
obs = "winds"
source = "All"
fmt = "csv"
sort_by = "dates"
qc = "on"

# Date range
start_date = datetime(2021, 1, 1)
end_date = datetime(2022, 12, 31)

# Function to generate URL
def generate_url(start, stop):
    return (f"https://data.gcoos.org/get_gcoos_data.php?"
            f"bbox={bbox}&start={start}&stop={stop}&obs={obs}&"
            f"source={source}&fmt={fmt}&sortBy={sort_by}&qc={qc}")

# Function to download and save data
def download_data(start, stop, file_prefix):
    url = generate_url(start, stop)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_prefix + '.csv', 'wb') as file:
            file.write(response.content)
        print(f"Data for {start} to {stop} downloaded and saved as '{file_prefix}.csv'")
    else:
        print(f"Failed to download data for {start} to {stop}. Status code: {response.status_code}")
        print("Response content:", response.text)

# Generate and download data in chunks of up to 3 days
def download_data_in_chunks(start_date, end_date, max_days=3):
    current_start = start_date
    while current_start <= end_date:
        current_end = min(current_start + timedelta(days=max_days - 1), end_date)
        file_prefix = f"wind_data_{current_start.strftime('%Y%m%d')}_{current_end.strftime('%Y%m%d')}"
        download_data(current_start.strftime('%Y-%m-%dT00:00:00Z'),
                      current_end.strftime('%Y-%m-%dT23:59:59Z'),
                      file_prefix)
        current_start = current_end + timedelta(days=1)

# Combine CSV files into one
def combine_csv_files(output_file='wind_data_combined.csv'):
    import glob
    all_files = glob.glob("wind_data_*.csv")
    combined_df = pd.concat([pd.read_csv(f) for f in all_files])
    combined_df.to_csv(output_file, index=False)
    print(f"Combined data saved as '{output_file}'")

# Main workflow
download_data_in_chunks(start_date, end_date)
combine_csv_files()


Data for 2021-01-01T00:00:00Z to 2021-01-03T23:59:59Z downloaded and saved as 'wind_data_20210101_20210103.csv'
Data for 2021-01-04T00:00:00Z to 2021-01-06T23:59:59Z downloaded and saved as 'wind_data_20210104_20210106.csv'
Data for 2021-01-07T00:00:00Z to 2021-01-09T23:59:59Z downloaded and saved as 'wind_data_20210107_20210109.csv'
Data for 2021-01-10T00:00:00Z to 2021-01-12T23:59:59Z downloaded and saved as 'wind_data_20210110_20210112.csv'
Data for 2021-01-13T00:00:00Z to 2021-01-15T23:59:59Z downloaded and saved as 'wind_data_20210113_20210115.csv'
Data for 2021-01-16T00:00:00Z to 2021-01-18T23:59:59Z downloaded and saved as 'wind_data_20210116_20210118.csv'
Data for 2021-01-19T00:00:00Z to 2021-01-21T23:59:59Z downloaded and saved as 'wind_data_20210119_20210121.csv'
Data for 2021-01-22T00:00:00Z to 2021-01-24T23:59:59Z downloaded and saved as 'wind_data_20210122_20210124.csv'
Data for 2021-01-25T00:00:00Z to 2021-01-27T23:59:59Z downloaded and saved as 'wind_data_20210125_202101

KeyboardInterrupt: 